- Sentence Extraction

In [ ]:
import pandas as pd
import Example
import sqlite3
# 신조어 목록 불러오기
df = pd.read_excel('new_words_var.xlsx')
new_words = df[df.y == 1].iloc[:,1].drop_duplicates(keep='first')

# 전체 데이터 합치기
conn = sqlite3.connect('Total_Ent.db')
df1 = pd.read_sql('SELECT head FROM head', conn)
conn.close()
conn = sqlite3.connect('HP.db')
df2 = pd.read_sql('SELECT head FROM head', conn)
conn.close()
head = df1.append(df2)

# 불용어처리 없는 sentence 추출
sent = Example.extract_sent(head, new_words)
sent = pd.DataFrame.from_dict(sent, orient='index')
conn = sqlite3.connect('sent.db')
sent.to_sql('sent', conn)

- Generating Model using subtokenizer

In [2]:
import sqlite3
import pandas as pd
from collections import defaultdict
import Example
from sklearn.model_selection import train_test_split
conn = sqlite3.connect('sent.db')
temp = pd.read_sql('SELECT * FROM sent', conn)

sent = list()
for _ in temp['0']:
    sent.extend(_.split('  '))

In [14]:
sgen = Example.generator_subtokenizer(sent)
X, y = sgen.create_var(sent)
sgen.model()

KeyboardInterrupt: 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=13)
hist = sgen.model.fit(X_train, y_train, epochs=100, batch_size=128, validation_date=(X_test,y_test), verbose=2)

In [ ]:
Dropout()

In [ ]:
Example.plot_history(['sgen':])

In [ ]:
sgen.sentence_generation('가성비', 10)

- Generating Model using ltokenizer

In [1]:
import sqlite3
import pandas as pd
from collections import defaultdict
import Example
from sklearn.model_selection import train_test_split
conn = sqlite3.connect('sent.db')
temp = pd.read_sql('SELECT * FROM sent', conn)

sent = list()
for _ in temp['0']:
    sent.extend(_.split('  '))
words = {_: 1.0 for _ in temp['index']}

In [ ]:
lgen = Example.generator_ltokenizer(sent, words)
lgen.create_vocab(sent)
X,y = lgen.create_var(sent)
lgen.model()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=13)
hist = lgen.model.fit(X_train, y_train, epochs=100, batch_size=128, validation_date=(X_test,y_test), verbose=2)

In [ ]:
lgen.sentence_generation('가성비', 10)